# To do

- One hot encode data (Complete)
- Rework windowing function (Complete)
- Fix Neural Network to work with OHE Data (Complete)
- Include SMAPE in Neural Network (Complete)
- Work on forecasting portion (Complete)
- Try another LSTM Layer (Complete. Didn't really improve the model)
- Increase Window Size (Complete. Didn't really improve the model)
- Expand dataset so that it includes all stores and items (500 trendlines) (Complete)
- Optimize neural network. Consider using Conv1D layer? Another LSTM layer? (Complete)
- Maybe try an encoder-decoder system
- Run and submit data

# Introduction

This is a demand forecaster for the kaggle competition at https://www.kaggle.com/c/demand-forecasting-kernels-only using a LSTM neural network

In [1]:
## Imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from pandas.plotting import register_matplotlib_converters
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

import torch as torch
import torch.nn as nn

In [2]:
## Import data

path_in = './data/'
train_cols = ['date', 'store', 'item', 'sales']
train_dtypes = {'date': 'str', 'store': 'int', 'item': 'int', 'sales': 'int'}
parse_dates = ['date']

test_cols = ['date', 'store', 'item']
test_dtypes = {'date': 'str', 'store': 'int', 'item': 'int'}


train = pd.read_csv(path_in + 'train.csv', dtype = train_dtypes, parse_dates = parse_dates)
test = pd.read_csv(path_in + 'test.csv', dtype = test_dtypes, parse_dates = parse_dates)

In [3]:
train.sample(10)

,date,store,item,sales
396838,2014-08-20,8,22,102
407927,2014-12-31,4,23,17
28278,2015-06-08,6,2,44
555868,2015-02-04,5,31,15
73182,2013-05-23,1,5,24
127429,2016-12-06,10,7,49
753311,2015-09-27,3,42,62
669693,2016-10-09,7,37,23
721017,2017-04-23,5,40,23
740905,2016-10-07,6,41,7


So there are 10 stores with 50 different products. The sales for each item range between 0 and 231, but they are mainly around the 30-70 range. There is five years worth of sales data (2013-2017) to train on and the goal is the predict the sales for the next 3 months.

## Preprocessing
Here we create a windowed dataset to preprocess the data

In [4]:
# Include other date information to be processed
train['day_of_week'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year - 2013 # Subtract earliest year to help normalize this information

In [5]:
# One Hot Encode data
columns_OHE = train.columns[((train.columns != 'sales') & (train.columns != 'date')) & (train.columns != 'year')]
train_OHE = pd.get_dummies(train, columns = columns_OHE)

In [6]:
OHE_cols = train_OHE.columns

In [7]:
def create_data_windows(df, width):
    """
    Function: Takes in the DataFrame, df, and splits it into rolling windows of size width to be used in a neural network.
    It outputs a dataframe and a target dataframe, which is the sales number to be predicted. The other information such
    as year, day of the week, are all taken from the target.
    Inputs: df - dataframe with columns [date, store, item, sales]
            width - the with of the window including the last sales data which will be the target
    Outputs: df_windowed - dataframe with the day of the week of the last day of the window, the store, and item numbers
                            along with the windowed sales data
             df_target - dataframe that contains the target date and target sales
    """
    # Retrieve column names that are not date nor sales
    other_columns = df.columns[((df.columns != 'sales') & (df.columns != 'date'))]
    
    # Initalize output datasets
    windowed_data = np.empty([len(df)-width+1, width - 1])
    df = df.reset_index(drop = True)
    
    # Create windows. E.g. [1, 2, 3, 4, 5, 6, 7] -> [[1,2,3,4], [2,3,4,5], [3,4,5,6]]
    # Also creates targets which would be [[5], [6], [7]] in the above case
    for i in range(len(df)-width+1):
        windowed_data[i] = list(df.iloc[i:i+width-1]['sales'])
        
        
    # Create a DataFrame to contain the windowed data
    windowed_columns = list(map(str,(range(1,width))))
    
    df_temp = pd.DataFrame(data = windowed_data, 
                               columns = windowed_columns,
                               dtype = 'int')
    # Create a DataFrame containing store, year, etc. information and combine with windowed data
    df_other = df.loc[width-1::, other_columns].reset_index(drop = True)
    
    df_windowed = pd.concat([df_temp, df_other], axis = 1)
    
    # Create target dataset
    df_target = pd.DataFrame(data = df.loc[width-1::, ['date', 'sales']]).reset_index(drop = True)
        
    return df_windowed, df_target

In [8]:
# Optional: Using only a part of the data set. Otherwise use stores 1-10 and items 1-50
stores = list(range(1, 11))
items = list(range(1,51))

In [9]:
# Windowing Data
window_size = 30

df = pd.DataFrame()
df_target = pd.DataFrame()

for store_num in stores:
    for item_num in items:
        # Calculate windows
        sample = train_OHE.loc[(train['store'] == store_num) & (train['item'] == item_num) & (train['year'] <= 3)]
        df_temp, df_target_temp = create_data_windows(sample, window_size + 1)
        
        # Append to dataset
        df = pd.concat([df, df_temp], axis = 0)
        df_target = pd.concat([df_target, df_target_temp], axis = 0)

In [10]:
# Train-test split
train_df, val_df, train_target_df, val_target_df = train_test_split(df, df_target, test_size=0.1, shuffle = True)

In [11]:
train_df

,1,2,3,4,5,6,7,8,9,10,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
1419,63,47,57,50,72,63,61,56,40,36,...,0,0,0,0,0,0,0,0,0,1
547,40,31,45,47,48,38,36,44,41,39,...,0,0,0,0,0,1,0,0,0,0
153,23,22,35,26,35,23,45,26,31,18,...,0,0,0,0,1,0,0,0,0,0
372,23,27,37,44,41,20,20,33,34,27,...,0,0,0,0,0,0,0,0,0,0
980,68,69,77,68,84,77,52,58,57,77,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,30,58,71,64,48,74,90,41,56,66,...,0,0,0,0,1,0,0,0,0,0
485,66,64,90,101,50,65,59,68,70,75,...,0,0,1,0,0,0,0,0,0,0
880,122,90,80,99,122,106,128,125,77,122,...,0,0,0,1,0,0,0,0,0,0
1255,36,41,44,41,25,27,42,35,36,43,...,0,0,0,0,1,0,0,0,0,0


In [12]:
train_target_df

,date,sales
1419,2016-12-20,39
547,2014-08-01,31
153,2013-07-03,27
372,2014-02-07,52
980,2015-10-08,78
...,...,...
545,2014-07-30,53
485,2014-05-31,74
880,2015-06-30,94
1255,2016-07-09,40


Turn dataframes into tensors to prepare them to be fed into neural network in pytorch

In [13]:
training_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(np.array(train_df)), torch.FloatTensor(np.array(train_target_df['sales'])))
validation_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(np.array(val_df)), torch.FloatTensor(np.array(val_target_df['sales'])))

# Neural Network

Initalize Neural Network

In [14]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
        nn.init.constant_(m.bias,0)
        
def SMAPE(y_true, y_pred):
    """ Returns the standard mean absolute percent error in %
    """
    return np.mean(200*np.abs(y_pred - y_true)/(np.abs(y_pred)+np.abs(y_true)))
        
class Net(nn.Module):
    def __init__(self, num_cols, window_length, n_hidden_1=32, n_hidden_2=32, n_hidden_3=32, n_hidden_4 = 32, D_out=1):
        super().__init__()
        
        self.window_length = window_length
        self.num_cols = num_cols
        
        # LSTM Layer
        self.lstm1 = nn.LSTM(1, n_hidden_1, bidirectional = False, batch_first = True)
        
        # Densely Connected Layers
        self.fc1 = nn.Linear(n_hidden_1 + num_cols - self.window_length, n_hidden_2)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(n_hidden_2, n_hidden_3)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(n_hidden_3, D_out)
        self.out_act = nn.ReLU()
        
    def forward(self, x):
        # Split input tensor. x_seq will be fed into the lstm.
        x_seq, x_info = x[:,:self.window_length], x[:,self.window_length:]
        
        # LSTM layer
        x_seq = torch.unsqueeze(x_seq, 2)
        lstm_out, (h, c) = self.lstm1(x_seq)
        
        # Combine the extra info to the lstm results using the output of the last lstm neuron
        lstm_output = torch.squeeze(lstm_out[:,-1,:], dim = 1)
        
        combined_out = torch.cat([lstm_output, x_info], 1)
        
        # Fully connected layer 1
        fc1_out = self.fc1(combined_out)
        fc1_out = self.relu1(fc1_out)
        
        # Fully connected layer 2
        fc2_out = self.fc2(fc1_out)
        fc2_out = self.relu2(fc2_out)
        
        # Output layer
        y = self.out_act(self.fc3(fc2_out))
        
        # Squeeze to remove extra dimensions of size 1
        return torch.squeeze(y)

In [15]:
def train_model(data_set, model, criterion, train_loader, validation_loader, optimizer, epochs=10):
    model.train()
    loss_accuracy = {'training_loss':[], 'validation_loss':[],'validation_smape':[], 'validation_precision':[], 'validation_recall':[]}
    
    training_size = (len(list(train_loader))-1)*len(list(train_loader)[0][0]) + len(list(train_loader)[-1][0])
    
    for epoch in range(epochs):
        #clear_output(wait=True)
        print("Epoch {} / {}\n=============".format(epoch+1, epochs))
            
        train_smape_sum = 0
        train_loss_sum = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            ## Forward pass
            yhat = model(x)
            ## Compute loss
            train_loss = criterion(yhat, y)
            train_loss_sum += train_loss.item() * len(x)
            ## Compute gradient in backward pass
            train_loss.backward()
            ## Update weights
            optimizer.step()
            
            train_smape_sum += SMAPE(y.numpy(), yhat.detach().numpy()) * len(x)
            
            
         
        ## Compute validation accuracy
        model.eval()
        correct = 0
        for x, y in validation_loader:
            yhat = net(x)
            val_loss = criterion(yhat, y)                     
            val_smape = SMAPE(y.numpy(), yhat.detach().numpy())
            
        # Calculate loss values
        train_loss_avg = train_loss_sum / training_size
        train_smape = train_smape_sum / training_size
        
        # Append values to loss_accuracy to save results
        loss_accuracy['training_loss'].append(train_loss_avg)
        
        loss_accuracy['validation_loss'].append(val_loss.item())
        loss_accuracy['validation_smape'].append(val_smape)
        
        
        ## Print training loss and accuracy, and validation accuracy
        
        print("Training mean squared error: {:.3f} | Training SMAPE: {:.2f}\nValidation mean squared error: {:.3f} | Validation SMAPE: {:.2f}".format(
            train_loss_avg, train_smape, val_loss.item(), val_smape))
        
        model.train()
        
        ## Add precision and recall
        
    print("Training complete!")
                
    return loss_accuracy

In [16]:
learning_rate = 0.0005

## Network dimensions
num_cols = len(train_df.columns)
n_hidden_1 = 32
n_hidden_2 = 128
n_hidden_3 = 64
D_out = 1
batch_size = 512
reg_lambda = 0.005
momentum_coef = 0.9
dropout_percent = 0.0

## Load data
train_loader = torch.utils.data.DataLoader(dataset=training_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset), shuffle=False)

## Initialize model
net = Net(num_cols, window_size, n_hidden_1, n_hidden_2, n_hidden_3, D_out)
net.apply(weights_init)


optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay = reg_lambda)
criterion = nn.MSELoss()

In [17]:
epochs = 12
## Train the model
loss_accuracy = train_model(training_dataset, net, criterion, train_loader, validation_loader, optimizer, epochs=epochs)

Epoch 1 / 12
Training mean squared error: 257.637 | Training SMAPE: 23.24
Validation mean squared error: 55.865 | Validation SMAPE: 13.06
Epoch 2 / 12


KeyboardInterrupt: 

In [ ]:
## Plots
fig = plt.figure(1)
plt.plot(loss_accuracy['training_loss'], color="red")
plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss [-]")

fig = plt.figure(2)
plt.plot(loss_accuracy['validation_smape'], color="blue")
plt.title("Validation SMAPE")
plt.xlabel("Epoch")
plt.ylabel("Validation SMAPE [%]")

# Forecasting

In [ ]:
def model_forecast(model, df, start_date, end_date, store_num, item_num, col_names):
    """
    This function creates a loop to forecast sales data into the future.
    Inputs: model: The trained neural network model (pytorch model)
            df: The dataframe containing the known past sequence data (dataframe)
            start_date: The first forecast date (datetime string)
            end_date: The last forecast date (datetime string)
            store_num: the store number (int)
            item_num: the item num (int)
            col_names: the names of the columns fed into the neural network to ensure that they are ordered properly (list of strings)
    """
    
    model.eval() # Ensure that the model is in evaluation mode
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    columns_OHE = df.columns[((train.columns != 'sales') & (train.columns != 'date')) & (train.columns != 'year')]
    
    #Initialize data
    current_date = start_date
    sales_seq = list(df.loc[(df['date'] >= (start_date - datetime.timedelta(days = window_size)))
                               & (df['date'] < start_date)
                               & (df['store'] == store_num)
                               & (df['item'] == item_num),'sales'])

    predictions = pd.DataFrame({'date': [], 'sales': [], 'store': [], 'item': []})
    
    # Count number of times the loop has to run
    dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

    for i in range(len(dates)):
        # Initialize store, date, and item data inputs to neural network
        df_misc_info = pd.DataFrame({'date': [current_date], 'item': [item_num], 'store': [store_num]})
        df_misc_info['date'] = pd.to_datetime(df_misc_info['date'], errors = 'coerce')
        df_misc_info['day_of_week'] = df_misc_info['date'].dt.dayofweek
        df_misc_info['month'] = df_misc_info['date'].dt.month
        df_misc_info['year'] = df_misc_info['date'].dt.year - 2013

        # Initialize sequential data
        windowed_columns = list(map(str,(range(1,window_size+1))))
        df_seq = pd.DataFrame(data = [sales_seq], 
                                   columns = windowed_columns,
                                   dtype = 'int')

        # Combine input data and ensure columns match
        df_input = pd.concat([df_seq, df_misc_info], axis = 1)
        df_input_OHE = pd.get_dummies(df_input, columns = columns_OHE)
        df_input_OHE = df_input_OHE.reindex(columns = col_names).fillna(0)

        # Forecast sales for next day
        x = Variable(torch.FloatTensor(np.array(df_input_OHE)))
        y = float(model(x))

        # Append date and data
        predictions = predictions.append({'date': current_date, 'sales': y,
                                          'store': store_num, 'item': item_num}, ignore_index = True)

        # Update sequence and date
        sales_seq[:-1] = sales_seq[1::]
        sales_seq[-1] = y

        current_date = current_date + datetime.timedelta(days = 1)
    
    return predictions

In [ ]:
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2017-03-31')

predictions = pd.DataFrame()

for store_num in stores:
    for item_num in items:
        predict_temp = model_forecast(net, train, start_date, end_date, store_num, item_num, df.columns)
        predictions = pd.concat([predictions, predict_temp], axis = 0)

In [ ]:
store_num = 5
item_num = 8

predictions.loc[(predictions['store'] == store_num) & (predictions['item'] == item_num)].set_index(keys = ['date']).sort_index(ascending=True)['sales'].plot()
val_set = train.loc[(train['store'] == store_num) & (train['item'] == item_num) 
                    & (train['date'] >= start_date) & (train['date'] <= end_date)]
val_set.set_index(keys = ['date']).sort_index(ascending=True)['sales'].plot()


In [ ]:
val_set = train.loc[(train['store'].isin(stores)) & (train['item'].isin(items))
                    & (train['date'] >= start_date) & (train['date'] <= end_date)]

val_merged = val_set.merge(predictions, on = ['store', 'item', 'date'])

SMAPE(np.array(val_merged['sales_x']),val_merged['sales_y'])

In [ ]:
training_dataset[0]

In [20]:
print(train_df.columns.tolist())

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 'year', 'store_1', 'store_2', 'store_3', 'store_4', 'store_5', 'store_6', 'store_7', 'store_8', 'store_9', 'store_10', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7', 'item_8', 'item_9', 'item_10', 'item_11', 'item_12', 'item_13', 'item_14', 'item_15', 'item_16', 'item_17', 'item_18', 'item_19', 'item_20', 'item_21', 'item_22', 'item_23', 'item_24', 'item_25', 'item_26', 'item_27', 'item_28', 'item_29', 'item_30', 'item_31', 'item_32', 'item_33', 'item_34', 'item_35', 'item_36', 'item_37', 'item_38', 'item_39', 'item_40', 'item_41', 'item_42', 'item_43', 'item_44', 'item_45', 'item_46', 'item_47', 'item_48', 'item_49', 'item_50', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'month_1', 'month_2', 'month_3', 'month_4', 'mont